# Delta Lake 0.7+ feature workarounds
Azure Synapse Analytics currently runs a fork of Delta Lake 0.6.x, which does not support all SQL commands and features available in Delta Lake 0.7+. This notebook contains .NET workarounds for these commands and features.


In [ ]:
// Generate some test data.
var df = spark.Sql("SELECT 'foo' as Col1, 'bar' as Col2");

## Creating managed tables (with or without partitions)


In [ ]:
spark.Sql("DROP TABLE IF EXISTS ManagedDeltaTable");
spark.Sql("DROP TABLE IF EXISTS ExternalDeltaTable");
spark.Sql("DROP TABLE IF EXISTS PartitionedManagedDeltaTable")

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// CREATE TABLE tableName USING DELTA

df.Write().
    Format("delta").
    SaveAsTable("ManagedDeltaTable");

In [ ]:
var externalTablePath = "/tutorial/delta/externaltable";
df.Write().
    Format("delta").
    Mode("overwrite").
    Save(externalTablePath);
spark.Sql($"CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '{externalTablePath}'");

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// CREATE TABLE tableName USING DELTA PARTITIONED BY (...)

df.Write().
  Format("delta").
  Mode("append").
  PartitionBy("Col1").
  Option("__partition_columns", "[\"Col1\"]").
  SaveAsTable("PartitionedManagedDeltaTable");

## Reading from a storage path


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// SELECT * FROM delta.`/path/`

spark.Read().
    Format("delta").
    Load(externalTablePath).
    Show();

## Inserting from one table into another


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// INSERT INTO table1 SELECT * FROM table2

spark.Sql("SELECT * FROM ManagedDeltaTable").
    Write().
    Format("delta").
    Mode("append").
    Save(externalTablePath);

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// INSERT OVERWRITE table1 SELECT * FROM table2

spark.Sql("SELECT * FROM ManagedDeltaTable").
    Write().
    Format("delta").
    Mode("overwrite").
    Save(externalTablePath);

## Updating or deleting rows from a table


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// DELETE FROM tableName WHERE (...)

using Microsoft.Spark.Extensions.Delta;
using Microsoft.Spark.Extensions.Delta.Tables;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

var dt = DeltaTable.ForPath(externalTablePath);

dt.Delete("Col1 == 'foo'");

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// UPDATE tableName SET (...)

var describeExtended = spark.Sql("DESCRIBE EXTENDED ManagedDeltaTable");
display(describeExtended);

In [ ]:
// Get the path to the table in storage.
var managedTablePath = (string)describeExtended.
    Where("col_name == 'Location'").
    Select("data_type").
    Collect().
    First().
    Get(0);

// Construct the DeltaTable object from the path.
var managedTable = DeltaTable.ForPath(managedTablePath);

// Run the update command.
managedTable.Update(
        condition: Expr("Col1 == 'foo'"),
        set: new Dictionary<string, Column>(){{ "Col2", Lit("foobar") }});

In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// UPDATE delta.`/path/` WHERE (...)
DeltaTable.ForPath(externalTablePath).
    Update(
        condition: Expr("Col1 == 'foo'"),
        set: new Dictionary<string, Column>(){{ "Col2", Lit("foobar") }});

## Merging two tables


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// MERGE INTO table1
// USING table2
// ON (...)
// WHEN MATCHED THEN (...)
// WHEN NOT MATCHED THEN (...)

DeltaTable.ForPath(externalTablePath).As("ExternalTable").
  Merge(managedTable.As("ManagedTable").ToDF(), "ExternalTable.Col1 == ManagedTable.Col1").
  WhenMatched().
    Update(new Dictionary<string, Column>(){{ "Col2", Lit("This row matched") }}).
  WhenNotMatched().
    Insert(new Dictionary<string, Column>(){{ "Col2", Lit("This row did not match") }}).
  Execute();



## Changing the schema of a managed table.


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// ALTER TABLE tableName ADD COLUMNS (...)
// ALTER TABLE tableName CHANGE COLUMN (...)
// ALTER TABLE tableName REPLACE COLUMNS (...)

// Drop external table.
spark.Sql("DROP TABLE ExternalDeltaTable");

// Reconfigure the table using DataFrame APIs...

// Recreate the table.
df.Write().
    Format("delta").
    Mode("overwrite").
    Save(externalTablePath);
spark.Sql($"CREATE TABLE ExternalDeltaTable USING DELTA LOCATION '{externalTablePath}'");

## Configuring table properties


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// ALTER TABLE delta.`/path`
// SET TBLPROPERTIES(...)
// TBLPROPERTIES(
// delta.compatibility.symlinkFormatManifest.enabled=true)

// No workaround available.


In [ ]:
// Delta Lake 0.7+ SQL syntax: 
// 
// TBLPROPERTIES(delta.logRetentionDuration = "interval <interval>")
// TBLPROPERTIES(delta.deletedFileRetentionDuration = "interval <interval>")

// Can only set these globally.
spark.Conf().Set("spark.databricks.delta.properties.defaults.delta.logRetentionDuration", "interval 2 days");
spark.Conf().Set("spark.databricks.delta.properties.defaults.delta.deletedFileRetentionDuration", "interval 1 days");

In [ ]:
// SET spark.databricks.delta.commitInfo.userMetadata=”{custom metadata}” INSERT …

// df.write.format("delta")
//   .mode(...)
//   .option("userMetadata", "{custom metadata}")
//   .save(...)

// No workaround available for these.


## DeltaTable.forName()


In [ ]:
// Delta Lake 0.7+ syntax: 
// 
// DeltaTable.forName(tableName)

var managedTablePath = (string)describeExtended.
    Where("col_name == 'Location'").
    Select("data_type").
    Collect().
    First().
    Get(0);

DeltaTable.ForPath(managedTablePath);